In [50]:
# Source : https://blog.nycdatascience.com/student-works/project-3-web-scraping-company-data-from-indeed-com-and-dice-com/

# load the library
from bs4 import BeautifulSoup as Soup
import urllib, requests, re, pandas as pd
import lxml, time, html5lib


import urllib2 # Website connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'



# indeed.com url
#base_url = 'http://www.indeed.com/jobs?q=data+scientist&jt=fulltime&sort='
#base_url = 'http://www.indeed.fr/jobs?q=data+scientist&jt=fulltime&sort='

#base_url = (
#    "http://www.indeed.fr/emplois?as_and=data&as_any=science+scientist+analytics+"
#    "analyst+analyste+visualisation+visualization+mining+dataming+dataminer+miner+learning+apprentissage"
#    "&sort="
# )

l_base_url = []

l_base_url.append(
    "http://www.indeed.fr/emplois?as_and=data&as_any=science"
    "&l=Île-de-France"
    "&sort="
    )

l_base_url.append(
    "http://www.indeed.fr/emplois?as_and=data&as_any=scientist"
    "&l=Île-de-France"
    "&sort="
    )

l_base_url.append(
    "http://www.indeed.fr/emplois?data&as_any=analytics+analyst+analyste"
    "&l=Île-de-France"
    "&sort="
    )

l_base_url.append(
    "http://www.indeed.fr/emplois?data&as_any=visualisation+visualization"
    "&l=Île-de-France"
    "&sort="
    )

l_base_url.append(
    "http://www.indeed.fr/emplois?data&as_any=mining+dataming+dataminer"
    "&l=Île-de-France"
    "&sort="
    )

l_base_url.append(
    "http://www.indeed.fr/emplois?data&as_any=machine+learning+apprentissage"
    "&l=Île-de-France"
    "&sort="
    )

pd.set_option('max_colwidth',500)    # to remove column limit (Otherwise, we'll lose some info)

start_ts = time.strftime("%Y%m%d-%H%M%S")


In [51]:
# Taken from http://nbviewer.jupyter.org/github/jmsteinw/Notebooks/blob/master/IndeedJobs.ipynb#
def text_cleaner(website):
    '''
    This function just cleans up the raw html so that I can look at it.
    Inputs: a URL to investigate
    Outputs: Cleaned text only
    '''
    try:
        site = urllib2.urlopen(website).read() # Connect to the job posting
    except: 
        return   # Need this in case the website isn't there anymore or some other weird connection problem 
    
    soup_obj = Soup(site) # Get the html from the site
    
    if len(soup_obj) == 0: # In case the default parser lxml doesn't work, try another one
        soup_obj = Soup(site, 'html5lib')
    
    
    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    
    

    text = soup_obj.get_text() # Get the text from this

    

    lines = (line.strip() for line in text.splitlines()) # break into lines

    
    
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    

    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    
    # Now clean out all of the unicode junk (this line works great!!!)
    
    
    # Need this as some websites aren't formatted
    #try:
        #text = text.decode('unicode_escape').encode('utf-8', 'ignore')
        
    #except:                                                            # in a way that this works, can occasionally throw
        #return                                                         # an exception
   
    
    #text = re.sub("[^a-zA-Z+3éèàếïîöô]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                             # Also include + for C++
    #text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    
    #text = text.lower().split()  # Go to lower case and split them apart
    

    #stop_words = set(stopwords.words("english")) # Filter out any stop words
    #stop_words = set(stopwords.words("french")) # Filter out any stop words
    #text = [w for w in text if not w in stop_words]
    
    
    
    #text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                           # or not on the website)
    
    return text

In [52]:
#%debug
# 10 (unsponsored) listings / page
# TODO : change moronic for loop to stop when no more pages

def get_listings(base_url):
    
    sort_by = 'date'          # sort by data
    start_from = '&start='    # start page number

    
    

    # Get the number of jobs
    url = "%s%s" % (base_url, sort_by) # get full url 
    target = Soup(urllib.urlopen(url), "lxml")    

    # Now find out how many jobs there were
    num_jobs_area = target.find(id = 'searchCount').string.encode('utf-8') # Now extract the total number of jobs found
                                                                         # The 'searchCount' object has this

    job_numbers = re.findall('\d+', num_jobs_area) # Extract the total jobs found from the search result


    if len(job_numbers) > 3: # Have a total number of jobs greater than 1000
        total_num_jobs = (int(job_numbers[2])*1000) + int(job_numbers[3])
    else:
        total_num_jobs = int(job_numbers[2]) 

    print 'There were', total_num_jobs, 'jobs found' # Display how many jobs were found

    num_pages = total_num_jobs/10 # This will be how we know the number of times we need to iterate over each new
                                  # search result page

    df = pd.DataFrame()   # create a new data frame
    job_descriptions = []

    #for page in range(1,101): # page from 1 to 100 (last page we can scrape is 100)
    for page_nb in range(1,num_pages + 1):
        print("page %d" % (page_nb))

        page = (page_nb-1) * 10  
        url = "%s%s%s%d" % (base_url, sort_by, start_from, page) # get full url 
        target = Soup(urllib.urlopen(url), "lxml")     

        targetElements = target.findAll('div', attrs={'class' : '  row  result'}) # we're interested in each row (= each job)

        job_nb = 1
        # trying to get each specific job information (such as company name, job title, urls, ...)
        for elem in targetElements: 
            print('job nb %d' %(job_nb))

            #comp_name = elem.find('span', attrs={'itemprop':'name'}).getText().strip()
            #job_title = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
            #job_addr = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()

            found_elem = elem.find('span', attrs={'itemprop':'name'})
            if found_elem is not None:            
                comp_name = found_elem.getText().strip()
                comp_link_overall = found_elem.find('a')

            found_elem = elem.find('a', attrs={'class':'turnstileLink'})
            if found_elem is not None:            
                job_title = found_elem.attrs['title']

            found_elem = elem.find('span', attrs={'itemprop':'addressLocality'})
            if found_elem is not None:            
                job_addr = found_elem.getText()

            home_url = "http://www.indeed.fr"
            job_link = "%s%s" % (home_url,elem.find('a').get('href'))        
            job_posted = elem.find('span', attrs={'class': 'date'}).getText()


            # Commented because of a bug in some listings and the field is not useful
            #if comp_link_overall is not None: # if company link exists, access it. Otherwise, skip.
            #    comp_link_overall = "%s%s" % (home_url, comp_link_overall.attrs['href'])
            #else: comp_link_overall = None

            

            # Get job description
            final_description = text_cleaner(job_link)
            if final_description: # So that we only append when the website was accessed correctly
                #print("job description OK")
                job_descriptions.append("page_nb%d, job_nb%d, %s" % (page_nb, job_nb, final_description))
                job_descr = "page_nb%d, job_nb%d, %s" % (page_nb, job_nb, final_description)
            else:
                job_descriptions.append("page_nb%d, job_nb%d, NA" % (page_nb, job_nb))
                job_descr = "page_nb%d, job_nb%d, NA" % (page_nb, job_nb)
                
                
            # add a job info to our data frame
            df = df.append({'page_nb' : page_nb, 'job_nb' : job_nb,
                            'comp_name': comp_name, 'job_title': job_title, 
                            'job_link': job_link, 'job_posted': job_posted,
                            'overall_link': comp_link_overall, 'job_location': job_addr,
                            'overall_rating': None, 'wl_bal_rating': None, 
                            'benefit_rating': None, 'jsecurity_rating': None, 
                            'mgmt_rating': None, 'culture_rating': None,
                            'job_descr': job_descr
                           }, ignore_index=True)


            job_nb = job_nb + 1
        # Sleep to avoid spamming the server and getting banned
        time.sleep(0.05)

    return df


In [53]:
l_df_job = []
for search in l_base_url:
    l_df_job.append(get_listings(search))

There were 992 jobs found
page 1
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 2
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 3
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 4
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 5
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 6
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 7
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 8
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 9
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 10
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page 11
job nb 1
job nb 2
job nb 3
job nb 4
job nb 5
job nb 6
job nb 7
job nb 8
job nb 9
page

In [54]:
# Save the result to CSV

for i in range(0, len(l_df_job)):
    fname_out = './data/indeed_companies_data_%s_%d.csv' % (start_ts, i)
    l_df_job[i].to_csv(fname_out, encoding='utf-8', sep = ';', quoting = True)
    
    # Save jop description to file
    #fname_job = './data/indeed_job_descr_%s_%d.csv' % (start_ts, i)
    #f_job = open(fname_job, 'w')

    #for item in job_descriptions:
      #print>>f_job, item


